In [1]:
# Import packages
import numpy as np
import math
import random
import time
import csv
from scipy.stats import poisson
from joblib import Parallel, delayed, dump, load
from matplotlib import pyplot as plt

In [3]:
# Can collect objective values to visualise improvement in the schedule over time
global objVals
objVals = []

# Relevant for printing to file when collecting data for the surrogate model
# np.set_printoptions(linewidth=2000)
# f = open('data/' + '28-04-2022_heuristic', 'w')
# global writer
# writer = csv.writer(f)

# Loading in of the ML models from files
# model = load('testing_model.joblib')
# model = load('testing_model_rnn.joblib')

## Setup problem definition

## Test case

- T = 4h
- d = 5 min, the size of an interval
- beta = 9 minutes, service time (model by the exponential distribution)
- N = 24 patients
- I = T/d, the number of intervals

In [4]:
"""Schedule class responsible for making the initial schedule with a sensible default layout
    """


class schedule:
    def __init__(self, beta: int, I: int, d: int, N: int):
        self.beta = beta  # = 1/μ : average service time
        self.I = I  # number of intervals
        self.d = d  # length of interval
        self.N = N  # total number of patients
        # schedule with x[t] as number of
        self.x: list[int] = np.repeat(0, self.I)
        # patients scheduled at the start of
        # interval t, t = 1,...,T; initially
        # set at zero

    def reset_schedule(self):
        for t in range(self.I):
            self.x[t] = 0

    def make_random_schedule(self, min_x: int, max_x: int, step: int = 1):
        n = self.N
        for i in range(0, self.I, step):
            if n >= 0:
                r = random.randint(min_x, max_x)
                self.x[i] = min(r, n)
                n = n - r
                print(n, self.x[i])
            else:
                return

    def make_initial_schedule(self):
        for i in range(self.N):
            t = round(i*self.I / self.N)
            if i > self.I:
                i -= 1
            self.x[t] += 1


## Functions

In [6]:
"""Implementation of local search, moves patient around a schedule, adopting any schedules that are an improvement
    """


def simpleSearch(x, beta, precision, limit, v, n, no_show, I, d, alpha_W, alpha_I, alpha_T, results, p_plus, p_min):
    for m in range(0, I-1):
        k = I-1
        while k >= 0:
            x_current = x.copy()
            # Adding one vector is equivalent to moving the arrival of one patient from interval k to interval k+1
            if x_current[k] > 0:
                print(x_current, k)
                next_k = (k + m + 1) % I

                x_current[k] -= 1
                x_current[next_k] += 1

                # calcFromK = min(k, next_k)

                temp_results = calcResults(x_current, beta, precision, limit, v, n,
                                           no_show, I, d, 0, alpha_W, alpha_I, alpha_T, p_plus, p_min, 0)
                print(temp_results['objVal'], results['objVal'])

                if temp_results['objVal'] < results['objVal']:
                    x = x_current
                    results = temp_results
                    k = (k + 1) % I
                    p_plus = results['p_plus']
                    p_min = results['p_min']

                else:  # undo the previous move and move left
                    x_current[k] += 1
                    x_current[next_k] -= 1
                    k -= 1

            else:
                k -= 1

    return x, results


In [81]:
"""Implementation of a parallel local search, simultaneously calculates the objective values of potential schedules
    """


def simpleParallelSearch(x, beta, precision, limit, v, n, no_show, I, d, alpha_W, alpha_I, alpha_T, results, p_plus, p_min):
    k = I-1
    while k >= 0:
        x_current = x.copy()
        if x_current[k] > 0:  # Adding one vector is equivalent to moving the arrival of one patient from interval k to interval k+1
            next_k = (k + 1) % I

            possible_k = list(range(next_k, I)) + list(range(0, k))
            collectEvals = Parallel(n_jobs=11)(delayed(preCalcResults)(x_current, beta, precision, limit, v, n,
                                                                       no_show, I, d, 0, alpha_W, alpha_I, alpha_T, p_plus, p_min, k, next_k) for next_k in possible_k)

            oldObjVal = results['objVal']
            for e in collectEvals:
                # Relevant for writing schedule-objective values pairs to a data file
                # writer.writerow([np.array2string(e['schedule'], separator=', '), e['results']['objVal']])
                # if a schedule is better is replaces the current one
                if e['results']['objVal'] < results['objVal']:
                    results = e['results']
                    x = e['schedule']
                    p_plus = e['results']['p_plus']
                    p_min = e['results']['p_min']
            objVals.append(e['results']['objVal'])

            # if no better schedule is found, leave as is and move on
            if results['objVal'] == oldObjVal:
                k -= 1

        else:
            k -= 1

    return x, results


In [82]:
"""Local search modified to use a machine learning model as an evaluation metric for schedules
    """


def simpleSurrogateSearch(x, I, objVal):
    k = I-1
    while k >= 0:
        x_current = x.copy()
        if x_current[k] > 0:  # Adding one vector is equivalent to moving the arrival of one patient from interval k to interval k+1
            next_k = (k + 1) % I
            possible_k = list(range(next_k, I)) + list(range(0, k))
            possible_schedules = preCalcSurrogateResults(
                x_current, k, possible_k)
            collectEvals = model.predict(possible_schedules)

            oldObjVal = objVal
            for i, objv in enumerate(collectEvals):
                if objv < objVal:  # if a schedule is better is replaces the current one
                    objVal = objv
                    x = possible_schedules[i]
                    objVals.append(objv)
                    print(possible_schedules[i], objv)

            if objVal == oldObjVal:  # if no better schedule is found leave as is and move on
                k -= 1

        else:
            k -= 1

    return x, objVal


In [83]:
"""Function to calculate results for the parallel local search function, this is the function that is parallelised
    """


def preCalcResults(x_current, beta, precision, limit, v, n, no_show, I, d, eind, alpha_W, alpha_I, alpha_T, p_plus, p_min, k, next_k):
    x_currentCopy = x_current.copy()
    x_currentCopy[k] -= 1
    x_currentCopy[next_k] += 1
    calcFromK = min(k, next_k)
    return {'schedule': x_currentCopy, 'next_k': next_k, 'results': calcResults(x_currentCopy, beta, precision, limit, v, n, no_show, I, d, eind, alpha_W, alpha_I, alpha_T, p_plus, p_min, calcFromK)}


In [84]:
"""Function to generate possible schedules for use in the surrogate local search
    """


def preCalcSurrogateResults(x_current, k, possible_k):
    possible_schedules = []
    for next_k in possible_k:
        x_currentCopy = x_current.copy()
        x_currentCopy[k] -= 1
        x_currentCopy[next_k] += 1
        possible_schedules.append(x_currentCopy)
    return np.array(possible_schedules)


In [6]:
"""Distribution to calculate service time of patients
    p[i]= probability of serving the patient in i mins given that
    the average service time is beta.
    """


def calculate_p(beta, size, precision=0.999999):  # Poisson distribution
    k = 0
    p = []

    while sum(p) < precision:  # fill accurate values up to precision limit
        p.append(poisson.pmf(k, beta))
        k += 1

    while len(p) < size:  # fill the rest of the values with 0
        p.append(0)
    return p, k

### Test
calculate_p(1, 20)

([0.36787944117144233,
  0.36787944117144233,
  0.18393972058572114,
  0.06131324019524039,
  0.015328310048810101,
  0.00306566200976202,
  0.0005109436682936698,
  7.299195261338139e-05,
  9.123994076672672e-06,
  1.013777119630298e-06,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 10)

In [8]:
"""Calculates the exponential limit if higher than 100, else returns 100
    """


def calcExponentialLimit(mu):
    return int(max(mu+4*mu**0.5, 100))


In [9]:
"""Binomial coefficient function
    """


def binomCoeff(k, i):
    return math.factorial(k) / (math.factorial(k - i) * math.factorial(i))

    """Binomial probability mass function
  """


def binomPMF(k, i, m, add_v, no_show):
    return binomCoeff(k, m) * add_v[m][i] * (1 - no_show)**m * no_show**(k-m)


In [10]:
"""Calculates tardiness
    """


def calcTardiness(p_min, limit, I):
    tardiness = 0
    # print(p_min[I])
    for k in range(limit):
        tardiness += k * p_min[I][k]  # I+1
    return tardiness

    """Calculates the idle time
  """


def calcIdletime(I, d, tardiness, n, no_show, beta):
    return (I * d) + tardiness - (n * (1 - no_show) * beta)  # I-1?

    """Calculates the waiting time
  """


def calcWaitingtime(p_min, x, p, limit, I, n):
    w = np.zeros((I+1, n+1, limit+1))
    waitingtime = 0

    for t in range(0, I):
        if x[t] > 0:
            for k in range(limit):
                w[t][0][k] = p_min[t][k]
        if x[t] > 1:
            for i in range(1, x[t]+1):
                for k in range(limit+1):
                    for j in range(k+1):
                        w[t][i][k] += w[t][i-1][j] * p[k-j]

    for t in range(0, I):
        for i in range(0, x[t]):
            for k in range(limit+1):
                waitingtime += w[t][i][k] * k

    waitingtime /= n
    return waitingtime


In [11]:
"""Calculates v
    """


def calculate_v(p, beta, precision, precision_limit, n, d, no_show=0):
    count = precision_limit
    limit = calcExponentialLimit(beta*n) + 1
    v = np.zeros((n+1, limit+d))
    add_v = np.zeros((n+1, limit+d))

    add_v[0][0] = 1
    for k in range(1, n+1):
        limit = calcExponentialLimit(beta*k)
        i = 0
        sum_v = 0
        while sum_v < precision and i <= limit:
            z = 0
            while z <= count:
                add_v[k][i] += p[z] * add_v[k-1][i-z]
                z += 1
            sum_v += add_v[k][i]
            i += 1

    for k in range(n+1):
        i = 0
        sum_v = 0
        while sum_v < precision and i <= limit:
            for m in range(k+1):
                v[k][i] += binomPMF(k, i, m, add_v, no_show)
            sum_v += v[k][i]
            i += 1

    return v, limit


In [12]:
"""Calculates the probability arrays, p minus and p plus
"""


def calculateProbabilities(x, precision, limit, v, I, d, p_plus, p_min, t_start=0):
    p_min[t_start:, :] = 0
    p_plus[t_start:, :] = 0

    if p_min[0][0] == 0:
        t_start = 1
        # Constraint 1
        p_min[0][0] = 1

        # Constraint 2
        sum_p = 0
        i = 0
        while sum_p < precision and i <= limit:
            p_plus[0][i] = v[x[0]][i]
            sum_p += p_plus[0][i]
            i += 1

    for t in range(t_start, I+1):  # calculate p_min and p_plus iteratively
        # Constraint 3
        for k in range(d+1):
            # probability of amount of work = 0 just before the start of t equals the cummulative probablity of amount of work less or equal to duration of the interval just at the start of the previous interval, t-1
            p_min[t][0] += p_plus[t-1][k]
        # Constraint 4
        i = np.arange(1, limit+1)
        # probability of amount of work = i just before the start of interval t equals the probablity of amount of work exceeding the duration of the interval by i just at the start of the previous interval, t-1
        p_min[t][i] = p_plus[t-1][i+d]

        # Constraint 5
        if t != I:  # I or I+1
            for i in range(limit+1):
                for j in range(i+1):
                    p_plus[t][i] += p_min[t][j] * v[x[t]][i-j]

    return p_min, p_plus, limit


In [91]:
## OLD

# """Calculates the probability arrays, p minus and p plus
# """
# def calculateProbabilities(x, precision, limit, v, I, d, p_plus, p_min, t_start=0):
#   p_min[t_start:, :] = 0
#   p_plus[t_start:, :] = 0

#   if p_min[0][0] == 0:
#     t_start = 1
#     # Constraint 1
#     p_min[0][0] = 1

#     # Constraint 2
#     sum_p = 0
#     i = 0
#     while sum_p < precision and i <= limit:
#       p_plus[0][i] = v[x[0]][i]
#       sum_p += p_plus[0][i]
#       i += 1


#   for t in range(t_start, I+1): # calculate p_min and p_plus iteratively 
#     # Constraint 3
#     for k in range(d+1):
#       # probability of amount of work = 0 just before the start of t equals the cummulative probablity of amount of work less or equal to duration of the interval just at the start of the previous interval, t-1
#       p_min[t][0] += p_plus[t-1][k] 
#     # Constraint 4
#     for i in range(1,limit+1):
      
#       # probability of amount of work = i just before the start of interval t equals the probablity of amount of work exceeding the duration of the interval by i just at the start of the previous interval, t-1
#       p_min[t][i] = p_plus[t-1][i+d]

#     # Constraint 5
#     if t != I: # I or I+1
#       for i in range(limit+1):
#         for j in range(i+1):
#           p_plus[t][i] += p_min[t][j] * v[x[t]][i-j]

#   return p_min, p_plus, limit

In [13]:
"""Combines and calculates the different components of the objective value and returns all the results in a dictionary
    """


def calcResults(x, beta, precision, limit, v, n, no_show, I, d, eind, alpha_W, alpha_I, alpha_T, p_plus, p_min, k):
    tic = time.perf_counter()
    p_min, p_plus, limit = calculateProbabilities(
        x, precision, limit, v, I, d, p_plus, p_min, k)
    toc = time.perf_counter()
    probT = toc-tic

    # Tardiness calcs
    tic = time.perf_counter()
    tardiness = calcTardiness(p_min, limit, I)
    toc = time.perf_counter()
    tardT = toc-tic

    # Idle time calcs new array of given shape and type, filled with zeros.
    tic = time.perf_counter()
    idletime = calcIdletime(I, d, tardiness, n, no_show, beta)
    toc = time.perf_counter()
    idletimeT = toc-tic

    # Waiting time calcs
    tic = time.perf_counter()
    waitingtime = calcWaitingtime(p_min, x, p, limit, I, n)
    toc = time.perf_counter()
    waitingtimeT = toc-tic

    objVal = alpha_W*waitingtime + alpha_I*idletime + alpha_T*tardiness

    print(f"Schedule: {x},\nObjective value: {objVal},\nProb calculation time: {probT:.6f} sec,\nWaiting time (timer): {waitingtime} ({waitingtimeT:.6f} sec),\nIdle time (timer): {idletime} ({idletimeT:.6f} sec),\nTardiness (timer): {tardiness} ({tardT:.6f} sec)\n")

    # Collect into a dictionary
    results = {'p_min': p_min, 'p_plus': p_plus, 'waitingTime': waitingtime,
               'idleTime': idletime, 'tardiness': tardiness, 'objVal': objVal}

    if eind == 1:
        fracExcess = calcFracExcess(p_min, I)
        results['fracExcess'] = fracExcess

    return results


In [14]:
"""Calculates the frac excess
    """


def calcFracExcess(p_min, I):
    fracExcess = 0
    t = I+1
    for j in range(1, len(p_min[t])):
        fracExcess += p_min[t][j]
    fracExcess *= 100
    return fracExcess


## Tests

In [19]:
precision = 0.9999
n = 8  # number of patients
beta = 9  # average service time for a patient
T = 50  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals
no_show = 0
iend = 0

s = schedule(beta, I, d, n)
s.make_initial_schedule()
x = s.x

# size of p has to be at least as big as the limit value here
size = calcExponentialLimit(beta*n)+1
p, precision_limit = calculate_p(beta, size, precision)
v, limit = calculate_v(p, beta, precision, precision_limit, n, d, no_show)

alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4

p_plus = np.zeros((I+1, limit+d+1))
p_min = np.zeros((I+1, limit+d+1))

results = calcResults(x, beta, precision, limit, v, n, no_show,
                      I, d, 0, alpha_W, alpha_I, alpha_T, p_plus, p_min, 0)


Schedule: [1 1 1 0 1 1 1 0 1 1],
Objective value: 12.870365053945003,
Prob calculation time: 0.046893 sec,
Waiting time (timer): 9.798788006106475 (0.000933 sec),
Idle time (timer): 0.25141641917068114 (0.000002 sec),
Tardiness (timer): 22.25141641917069 (0.000046 sec)



In [20]:
simpleSearch(x, beta, precision, limit, v, n, no_show, I, d,
             alpha_W, alpha_I, alpha_T, results, p_plus, p_min)


[1 1 1 0 1 1 1 0 1 1] 9
Schedule: [2 1 1 0 1 1 1 0 1 0],
Objective value: 14.891708185850774,
Prob calculation time: 0.072906 sec,
Waiting time (timer): 15.243713525111673 (0.015635 sec),
Idle time (timer): -0.009628706989829539 (0.000002 sec),
Tardiness (timer): 21.990371293010174 (0.000054 sec)

14.891708185850774 12.870365053945003
[1 1 1 0 1 1 1 0 1 1] 8
Schedule: [1 1 1 0 1 1 1 0 0 2],
Objective value: 12.774425065391162,
Prob calculation time: 0.048359 sec,
Waiting time (timer): 9.229311054444683 (0.012424 sec),
Idle time (timer): 0.4711677393554794 (0.000002 sec),
Tardiness (timer): 22.471167739355483 (0.000048 sec)

12.774425065391162 12.870365053945003
[1 1 1 0 1 1 1 0 0 2] 9
Schedule: [2 1 1 0 1 1 1 0 0 1],
Objective value: 14.650842444418545,
Prob calculation time: 0.046187 sec,
Waiting time (timer): 14.621234339171174 (0.012144 sec),
Idle time (timer): 0.0039145145834567074 (0.000002 sec),
Tardiness (timer): 22.00391451458346 (0.000045 sec)

14.650842444418545 12.7744250653

(array([1, 1, 0, 1, 1, 0, 1, 1, 0, 2]),
 {'p_min': array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [1.15690521e-01, 9.10903190e-02, 1.17116124e-01, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         ...,
         [7.07660492e-03, 7.84115962e-03, 1.30748235e-02, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [1.12013419e-01, 4.51811666e-02, 5.33398728e-02, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [3.32180304e-08, 1.26078474e-07, 4.97598371e-07, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00]]),
  'p_plus': array([[1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [1.23409804e-04, 1.11068824e-03, 4.99809707e-03, ...,
        

# Main Function

In [21]:
"""Main Function
    """
precision = 0.9999
n = 24  # number of patients
beta = 9  # average service time for a patient
T = 4*60  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals

# precision = 0.9999
# n = 15 # number of patients
# beta = 9 # average service time for a patient
# T = 2*60 # total time
# d = 5 # interval size
# I = int(T/d) # number of intervals

no_show = 0
iend = 0

s = schedule(beta, I, d, n)
s.make_initial_schedule()
x = s.x
# x = np.zeros(I, dtype=int) # alternative x with all patients in the first timeslot
# x[0] = n

# size of p has to be at least as big as the limit value here
size = calcExponentialLimit(beta*n)+1
p, precision_limit = calculate_p(beta, size, precision)
v, limit = calculate_v(p, beta, precision, precision_limit, n, d, no_show)

alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4

p_plus = np.zeros((I+1, limit+d+1))
p_min = np.zeros((I+1, limit+d+1))

results = calcResults(x, beta, precision, limit, v, n, no_show,
                      I, d, 0, alpha_W, alpha_I, alpha_T, p_plus, p_min, 0)
objVals.append(results['objVal'])

x, results = simpleSearch(x, beta, precision, limit, v, n, no_show, I, d,
                          alpha_W, alpha_I, alpha_T, results, results['p_plus'], results['p_min'])
bestObjVal = results['objVal']
print(x, results)
x, results = simpleSearch(x, beta, precision, limit, v, n, no_show, I, d,
                          alpha_W, alpha_I, alpha_T, results, results['p_plus'], results['p_min'])
# Incase one iteration of moving things around is not enough
while results['objVal'] < bestObjVal:  # repeat until optimum is found
    bestObjVal = results['objVal']
    x, results = simpleSearch(x, beta, precision, limit, v, n, no_show, I, d,
                              alpha_W, alpha_I, alpha_T, results, results['p_plus'], results['p_min'])
print(x, results)

# Relevant for printing results to a file
# f.close()


Schedule: [1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 0 1 0 1 0 1 0],
Objective value: 7.472649552726008,
Prob calculation time: 1.544077 sec,
Waiting time (timer): 2.2829087524682325 (0.007525 sec),
Idle time (timer): 26.932476752897855 (0.000003 sec),
Tardiness (timer): 2.9324767528978555 (0.000118 sec)

[1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 0 1 0 1 0 1 0] 46
Schedule: [1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 0 1 0 1 0 0 1],
Objective value: 8.71054768421639,
Prob calculation time: 1.590730 sec,
Waiting time (timer): 2.2024861242878075 (0.008252 sec),
Idle time (timer): 29.04925539083544 (0.000002 sec),
Tardiness (timer): 5.049255390835446 (0.000120 sec)

8.71054768421639 7.472649552726008
[1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 0 1 0 1 0 1 0] 44
Schedule: [1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1

KeyboardInterrupt: 

In [ ]:
"""Plot the recorded objective values
    """
plt.plot(objVals)
# bottom axis should be iteration number where the best is updated, rather than amount of times it has been updated


In [ ]:
"""Alternative main function used for testing the surrogate model
    """
precision = 0.9999
n = 24  # number of patients
beta = 9  # average service time for a patient
T = 4*60  # total time
d = 5  # interval size
I = int(T/d)  # number of intervals

# precision = 0.9999
# n = 15 # number of patients
# beta = 9 # average service time for a patient
# T = 2*60 # total time
# d = 5 # interval size
# I = int(T/d) # number of intervals

s = schedule(beta, I, d, n)
s.make_initial_schedule()
x = s.x
# x = np.zeros(I, dtype=int) # alternative x with all patients at the beginning
# x[0] = n

objVal = model.predict(x.reshape(1, -1))[0]
objVals.append(objVal)

print(objVal)
x, objVal = simpleSurrogateSearch(x, I, objVal)
bestObjVal = objVal
print(x, objVal)

x, objVal = simpleSurrogateSearch(x, I, objVal)

while objVal < bestObjVal:  # repeat until optimum is found
    bestObjVal = objVal
    x, objVal = simpleSurrogateSearch(x, I, objVal)
print(x, objVal)


In [ ]:
"""Used to calculate the final actual objective value for the surrogate model result
    """
no_show = 0
iend = 0

# size of p has to be at least as big as the limit value here
size = calcExponentialLimit(beta*n)+1
p, precision_limit = calculate_p(beta, size, precision)
v, limit = calculate_v(p, beta, precision, precision_limit, n, d, no_show)

alpha_I = 0.2
alpha_T = 0.4  # patient doctor centric slider
alpha_W = 0.4

p_plus = np.zeros((I+1, limit+d+1))
p_min = np.zeros((I+1, limit+d+1))

results = calcResults(x, beta, precision, limit, v, n, no_show,
                      I, d, 0, alpha_W, alpha_I, alpha_T, p_plus, p_min, 0)
